# importing libraries

In [1]:
from utility.utility_general import *
import imp
from utility import util_plot
from utility import util_plot as Plots
import image_generator
imp.reload(util_plot)
imp.reload(Plots)

# pd.set_option('display.width', 1000)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_colwidth', 500)

from datetime import datetime
from joblib import Parallel, delayed
import tqdm
from ase.formula import Formula
from IPython.display import display, Markdown

from ase.formula import Formula

from pymatgen.ext.matproj import MPRester
api_key = "MCpqDZgh29W9r6X4Ru"
mpr: MPRester = MPRester(api_key)
    
import tensorflow as tf

import config


sky_line_path = data_path + '/skyline'

# Data collection
## Getting Supporting Information and converting it to text

In [ ]:
if not exists(sky_line_path + '/skyline_sup.txt'):
    # If the pdf doesn't exist:
    # https://advances.sciencemag.org/content/advances/suppl/2018/04/16/4.4.eaaq0148.DC1/aaq0148_SM.pdf
    if not exists(sky_line_path + 'skyline_sup.pdf'):
        import urllib.request
        print('the supporting info does not exist, downloading the info from the web')
        url = 'https://advances.sciencemag.org/content/advances/suppl/2018/04/16/4.4.eaaq0148.DC1/aaq0148_SM.pdf'
        urllib.request.urlretrieve(url, sky_line_path + 'skyline_sup.pdf')
    raw = parser.from_file(sky_line_path + 'skyline_sup.pdf')

    write_text(sky_line_path + 'skyline_sup.pdf', raw['content'])
txt = read_text(sky_line_path + 'skyline_sup.txt')

# Extracting all the information from the MP

In [2]:
amorphous = read_text(sky_line_path + '/aaq0148_Database S1.json')
amorphous = re.findall("\"[A-Z][A-Za-z0-9]*\": \[[\-0-9., ]+\]", amorphous)
compositions = [re.findall("\".*\"", i)[0][1:-1] for i in amorphous]
energies = np.array([[float(e) for e in re.findall("[\-0-9.]+", d.split(':')[1])] for d in amorphous],
                    dtype=object)
min_eng = [np.min(i) for i in energies]
ind = np.argsort(compositions)
compositions = np.array(compositions)[ind]
min_eng = np.array(min_eng)[ind]
energies = energies[ind]
save_var({'min_eng': min_eng, 'eng': energies, 'composition':compositions}, sky_line_path + '/row_variables.pkl')

In [23]:
data = [mpr.get_data(i) for i in tqdm.tqdm_notebook(compositions)]
save_var(data, 'tmp/mp-skyline_query.pkl')

In [8]:
data = load_var('tmp/mp-skyline_query.pkl')
data_flatten = [item for sublist in data for item in sublist]

df = pd.DataFrame({'query': data_flatten})

for col in ['material_id', 'pretty_formula', 'full_formula', 'task_ids', 
            'icsd_id', 'icsd_ids', 'e_above_hull', 'energy_per_atom']:
    df[col] = [i[col] for i in df['query']]
df['icsd_ids'] = [(None if len(i)==0 else i) for i in df['icsd_ids']]
    
df['sgn'] = [i['spacegroup']['number'] for i in df['query']]
df['hall'] = [i['spacegroup']['hall'] for i in df['query']]
df['full_formula_hill'] = [Formula(i).format('hill') for i in df['full_formula']]

df = df.drop_duplicates(subset=['sgn', 'full_formula_hill'], keep='first')
# df[df.duplicated(subset=['sgn', 'full_formula_hill'], keep='first')]
# df[df.duplicated(subset=['sgn', 'pretty_formula'], keep='first')]
df.to_csv('tmp/mp-skyline_query.csv')
df

,query,material_id,pretty_formula,full_formula,task_ids,icsd_id,icsd_ids,e_above_hull,energy_per_atom,sgn,hall,full_formula_hill
0,"{'energy': -147.85231451, 'energy_per_atom': -...",mp-755066,Al2O3,Al8O12,"[mp-859500, mp-858703, mp-755066, mp-1251182, ...",None,None,0.088742,-7.392616,14,-P 2ybc,Al8O12
1,"{'energy': -291.46625655, 'energy_per_atom': -...",mp-1247896,Al2O3,Al16O24,"[mp-1247896, mp-1818134]",None,None,0.194702,-7.286656,1,P 1,Al16O24
2,"{'energy': -724.93346081, 'energy_per_atom': -...",mp-1245196,Al2O3,Al40O60,"[mp-1245196, mp-1848370]",None,None,0.232023,-7.249335,1,P 1,Al40O60
5,"{'energy': -565.85781994, 'energy_per_atom': -...",mp-1247839,Al2O3,Al32O48,"[mp-1247839, mp-1865912]",None,None,0.541335,-7.073223,1,P 1,Al32O48
7,"{'energy': -298.57883609, 'energy_per_atom': -...",mp-2254,Al2O3,Al16O24,"[mp-672736, mp-661358, mp-2254, mp-555389, mp-...",None,"[84375, 94485, 161790, 247304]",0.016887,-7.464471,33,P 2c -2n,Al16O24
...,...,...,...,...,...,...,...,...,...,...,...,...
1331,"{'energy': -225.96485502, 'energy_per_atom': -...",mp-776386,ZrO2,Zr8O16,"[mp-776386, mp-1343477, mp-1809809]",None,None,0.088289,-9.415202,62,-P 2ac 2n,O16Zr8
1333,"{'energy': -56.80348487, 'energy_per_atom': -9...",mp-2574,ZrO2,Zr2O4,"[mp-665220, mp-656548, mp-659686, mp-673164, m...",None,"[291452, 66789, 93031, 92093, 92091, 157621, 6...",0.036244,-9.467247,137,P 4n 2n -1n,O4Zr2
1334,"{'energy': -28.30375563, 'energy_per_atom': -9...",mp-1565,ZrO2,Zr1O2,"[mp-657181, mp-657185, mp-657183, mp-2859, mp-...",None,"[291453, 291119, 185123, 186672, 647689, 92095...",0.068906,-9.434585,225,-F 4 2 3,O2Zr
1335,"{'energy': -114.0418938, 'energy_per_atom': -9...",mp-2858,ZrO2,Zr4O8,"[mp-670146, mp-675607, mp-660454, mp-2858, mp-...",None,"[291451, 291120, 80042, 80047, 80043, 62993, 8...",0.000000,-9.503491,14,-P 2ybc,O8Zr4


## Saving CIF files

In [9]:
makedirs(sky_line_path + '/cif_files', exist_ok=True)
df['path'] = None
for i, row in tqdm.tqdm_notebook(df.iterrows(), total=len(df)):
    path = sky_line_path + f'/cif_files/{row["material_id"]}.cif'
    write_text(path, row['query']['cif'])
    df.loc[i, 'path'] = path

In [10]:
df.drop(columns=['query', 'task_ids'], errors='ignore', inplace=True)
df.to_csv(sky_line_path + '/df.csv', index=False)

# Evaluation

In [5]:
df = pd.read_csv(sky_line_path + '/df.csv')

## By CNN

In [55]:
# cnn_dir = 'finalized_results/cnn-3-13-7-over'

# from tensorflow.keras.models import load_model

# trained_model = load_model(cnn_dir + '/model.h5')
# trained_model.load_weights(cnn_dir + '/weights0006.h5')


# df = pd.read_csv(sky_line_path + '/df.csv')
# df['img_path'] = df['path']
# df['y'] = 0
# generator = image_generator.ImageGeneratorDataFrame(df)

In [56]:
# yp = trained_model.predict(
#     generator, 
#     steps=len(generator),
#     verbose=1,
#     use_multiprocessing=True,
#     workers=32,
#     max_queue_size=10,
#                              )
# df['yp'] = yp.flatten()
# df['ypl'] = np.sign(np.sign(yp.flatten() - 0.5) + .5)
# df.to_csv(sky_line_path + '/df.csv')

1341/1341 [==============================] - 408s 303ms/step


## By CAE + MLP Classifier

In [12]:
from tensorflow.keras.models import load_model

encoder = load_model(config.cae_mlp_model_dir + '/encoder.h5', compile=False)
encoder.load_weights(config.cae_mlp_model_dir + '/encoder_weights.h5')

df['img_path'] = df['path']
df['y'] = 0

generator = image_generator.ImageGeneratorDataFrame(df)
lsr = encoder.predict(
    generator, 
    steps=len(generator),
    verbose=1,
    use_multiprocessing=True,
    workers=32,
    max_queue_size=10,
                             )

815/815 [==============================] - 272s 332ms/step


In [13]:
clf = load_var(config.cae_mlp_model_clf_dir + '/classifier_class.pkl')
yp = clf.predict_proba(lsr)
df['yp'] = yp.flatten()
df['ypl'] = np.sign(np.sign(yp.flatten() - 0.5) + .5)
df.to_csv(sky_line_path + '/df.csv', index=False)

# Convex hull calculations

In [14]:
df_all = pd.read_csv(sky_line_path + '/df.csv', index_col=0)
# skyline_vars = load_var(sky_line_path + '/variables.pkl')
skyline_vars = load_var(sky_line_path + '/row_variables.pkl')

compositions = skyline_vars['composition']
min_eng = skyline_vars['min_eng']
eng = skyline_vars['eng']

# df_e = df_all.sort_values('e_above_hull')
# df_e['diff'] = df_e['energy_per_atom'] - df_e['e_above_hull']
# [df_all[df_all['pretty_formula'] == c]['e_above_hull'].min() for c in compositions]
# df_e[df_e['pretty_formula'] == 'Al2O3']

min_convex_hull = []
for c in compositions:
#     c = compositions[0]
    df = df_all[df_all['pretty_formula'] == c]
    min_convex_hull.append(np.average(df['energy_per_atom'] - df['e_above_hull']))
min_eng_above_ground = min_eng - np.array(min_convex_hull)
min_eng_above_ground = {compositions[i]: min_eng_above_ground[i] for i in range(len(compositions))}
for i in range(len(eng)):
    for j in range(len(eng[i])):
        eng[i][j] -= min_convex_hull[i]
        
skyline_vars['min_eng_above_ground'] = min_eng_above_ground
skyline_vars['min_convex_hull'] = min_convex_hull
skyline_vars['eng_min_convex_hull'] = eng
save_var(skyline_vars, sky_line_path + '/variables.pkl')